**badNonBooleanValue.ipynb**

Invalid non-booeal annotation property value.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/badNonBooleanValue.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [5]:
import sys
from rdflib import ConjunctiveGraph, Namespace
import json
from collections import OrderedDict

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


https://neuroml.org/neuromlv2|https://neuroml-db.org/ does not look like a valid URI, trying to serialize this will break.


Define SPARQL query to retrieve ID, term, file extension of (EDAM Format) concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for ``is_deprecation_candidate`` & ``notRecommendedForAnnotation``below).

**NB.2:** Detects three annotation properties that require boolean values:
* ``<owl:deprecated>``
* ``<is_deprecation_candidate>``
* ``<notRecommendedForAnnotation>`` 

In [6]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?deprecated ?candidate ?notrecommended WHERE
{
?id rdfs:label ?term .
OPTIONAL {?id owl:deprecated ?deprecated .}
OPTIONAL {?id :is_deprecation_candidate ?candidate .}
OPTIONAL {?id :notRecommendedForAnnotation ?notrecommended} 
}
"""

# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

In [7]:
report.append('Invalid value (must be "true") found for elements of these concepts:')

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term']) 
    deprecated  = str(r['deprecated'])
    candidate = str(r['candidate'])
    notrecommended = str(r['notrecommended'])

    # "None" string is return from SPARQL query where value was not found
    if (deprecated != "None" and deprecated != "true"):
        errfound = True
        report.append('Value other than "true" for <deprecated> element::: ' + id +  ' (' + term + ') ' + deprecated)

    # "None" string is return from SPARQL query where value was not found
    if (candidate != "None" and candidate != "true"):
        errfound = True
        report.append('Value other than "true" for <is_deprecation_candidate> element::: ' + id +  ' (' + term + ') ' + candidate)

    # "None" string is return from SPARQL query where value was not found
    if (notrecommended != "None" and notrecommended != "true"):
        errfound = True
        report.append('Value other than "true" for <notRecommendedForAnnotation> element::: ' + id +  ' (' + term + ') '  + notrecommended)

Write report and return approriate value.

In [8]:
# Write report
# Use ordered dictionary to get consistent listing of properties in the JSON output
report_dict = OrderedDict

if errfound:
    report_dict = {
        'test_name': 'badNonBooleanValue',
        'status' : WARN,
        'reason' : report
    }
else:
    report_dict = {
        'test_name': 'badNonBooleanValue',
        'status' : NOERR,
        'reason' : '-'
    }

report_json = json.dumps(report_dict, indent=4)
print(report_json)

{
    "status": "NOERR",
    "test_name": "badNonBooleanValue",
    "reason": "-"
}
